# Ecommerce Logistics Analysis, Part 5
### Geographic Analysis


In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from table_audit import table_audit_function

%matplotlib inline

#Create context manager to handle connection to SQLite database and return dataframe from query
def run_query(q):
    with sqlite3.connect('data/ecommerce.db') as conn:
        return pd.read_sql_query(q,conn)

Below is a schema summarizing the relationships between the tables in the database I created in Part 1:
![title](images/schema_rearranged_2.jpg)

### Missing values investigation

In this section, dealing with latitude and longitude values of customers and sellers. 
Looking at shipments that were actually delivered (order_status='delivered')

In [2]:
pd.set_option('display.max_columns', 100)

q1 = '''
SELECT 
    c.*,
    g.geolocation_lat c_lat,
    g.geolocation_lng c_lng,
    merged.*,
    s.*,
    g2.geolocation_lat s_lat,
    g2.geolocation_lng s_lng
FROM (
    SELECT *
    FROM orders_modified om
    LEFT JOIN order_items_modified oim ON oim.order_id=om.order_id
    GROUP BY om.order_id
    ) as merged
LEFT JOIN customers c ON c.customer_id = merged.customer_id
LEFT JOIN geolocation g ON g.geolocation_zip_code = c.customer_zip_code_prefix
LEFT JOIN sellers s ON s.seller_id = merged.seller_id
LEFT JOIN geolocation g2 ON g2.geolocation_zip_code = s.seller_zip_code_prefix
'''

df1 = run_query(q1)
df1

OperationalError: unable to open database file

In [ ]:
table_audit_function(df1)

In [ ]:
q2 = '''
SELECT 
    c.*,
    g.geolocation_lat c_lat,
    g.geolocation_lng c_lng,
    merged.*,
    s.*,
    g2.geolocation_lat s_lat,
    g2.geolocation_lng s_lng
FROM (
    SELECT *
    FROM orders_modified om
    LEFT JOIN order_items_modified oim ON oim.order_id=om.order_id
    GROUP BY om.order_id
    ) as merged
LEFT JOIN customers c ON c.customer_id = merged.customer_id
LEFT JOIN geolocation g ON g.geolocation_zip_code = c.customer_zip_code_prefix
LEFT JOIN sellers s ON s.seller_id = merged.seller_id
LEFT JOIN geolocation g2 ON g2.geolocation_zip_code = s.seller_zip_code_prefix
WHERE merged.order_status='delivered' AND length(c_lat)>0 AND length(s_lat)>0
'''

df2 = run_query(q2)
df2

In [ ]:
table_audit_function(df2)

#### End of missing values investigation

In [ ]:
#Need to drop repeated columns: customer_id, order_id, seller_id
geo_df = df2

# df.columns.duplicated returns list of booleans, false if not repeated up to that point and true if repeated
# Use '~' to flip trues-> false and false-> trues
# End result is df w/ no repeated column names
geo_df = geo_df.loc[:,~geo_df.columns.duplicated()]

geo_df

In [ ]:
print('Number of orders: ', len(geo_df['order_id']))
print('Number of customers: ', len(geo_df['customer_unique_id'].unique()))
print('Number of sellers: ', len(geo_df['seller_id'].unique()))

In [ ]:
#from mpl_toolkits.basemap import Basemap
#from matplotlib.collections import LineCollection

#fig = plt.figure(figsize=(15,10))
#ax1 = fig.add_subplot(121)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
#map_customers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=80, llcrnrlon=-130, urcrnrlon=30)
#map_customers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)

#map_customers.drawstates(linewidth=0.5)
#map_customers.drawcoastlines(linewidth=0.5)
#map_customers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
#customers_x, customers_y = map_customers(list(geo_df['c_lng']), list(geo_df['c_lat']))


#3. Plot cartesian coordinates
#ax1.scatter(customers_x,customers_y,s=3, color='red')
#ax1.set_title('Customer Locations')


#ax2 = fig.add_subplot(122)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor

#map_sellers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=80, llcrnrlon=-130, urcrnrlon=30)
#map_sellers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)
#map_sellers.drawstates(linewidth=0.5)
#map_sellers.drawcoastlines(linewidth=0.5)
#map_sellers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
#sellers_x, sellers_y = map_sellers(list(geo_df['s_lng']), list(geo_df['s_lat']))


#3. Plot cartesian coordinates
#ax2.scatter(sellers_x,sellers_y,s=3,c='tab:orange')
#ax2.set_title('Seller Locations')
#plt.savefig('images/incorrect_location12345.JPG',bbox_inches='tight')
#plt.show()

![title](images/incorrect_map_location_2.JPG)

Observation: It looks like a few of the shipments went to customers in Portugal. Going to investigate this.

### Incorrect value in database investigation

Upon further investigation, the shipments that seem to travel to Portugal all have incorrect latitude/longitude coordinates: The zip codes for these locations are being convered to the wrong latitude/longitude coordinates. 

Note how 'porto trombetas' is a Brazilian city in the state of Para (PR) w/ a zip code prefix of 68275 which actually corresponds to a latitude & longitude of ____. 'areia branca dos assis' is a Brazilian city in the state of Parana w/ a zip code of 83810 which actually corresponds to a latitude & longitude of ____. 'ilha dos valadares' is a Brazilian city in the state of Parana w/ a zip code of 83252 which actually corresponds to a latitude & longitude of ____. It is odd that these latitude/longitude happened to be erroneously converted to latitude/longitude coordinates in Portugal considering both Brazil and Portugal are countries that predominantly speak Portugese.

![title](images/incorrect_data.JPG)

Will update SQL database w/ the correct latitude/longitude values for these shipments:

![title](images/update_83810.JPG)
![title](images/update_68275.JPG)
![title](images/update_83252.JPG)

In [ ]:
#Updated map

from mpl_toolkits.basemap import Basemap
from matplotlib.collections import LineCollection

fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(121)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
#map_customers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=80, llcrnrlon=-130, urcrnrlon=30)
map_customers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)

map_customers.drawstates(linewidth=0.5)
map_customers.drawcoastlines(linewidth=0.5)
map_customers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
customers_x, customers_y = map_customers(list(geo_df['c_lng']), list(geo_df['c_lat']))

#3. Plot cartesian coordinates
ax1.scatter(customers_x,customers_y,s=3)
ax1.set_title('Customer Locations')
ax2 = fig.add_subplot(122)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
map_sellers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)
map_sellers.drawstates(linewidth=0.5)
map_sellers.drawcoastlines(linewidth=0.5)
map_sellers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
sellers_x, sellers_y = map_sellers(list(geo_df['s_lng']), list(geo_df['s_lat']))

#3. Plot cartesian coordinates
ax2.scatter(sellers_x,sellers_y,s=3,c='tab:orange')
ax2.set_title('Seller Locations')
plt.savefig('images/customer_seller_locations.JPG',bbox_inches='tight')
plt.show()

#Note that some of the locations are still off: the location that looks to be in Argentina is actually Santa Rosa which is firmly in Brazil. Another case of wrong lat/lng values.

In [ ]:
import math as m

# Will estimate distances using haversine formula
# Is approximate b/c this calculates distances between two points on a sphere (Earth is not a perfect sphere)
# https://en.wikipedia.org/wiki/Haversine_formula

def haversine_dist(lat1,lng1,lat2,lng2):
    lat1_rad = lat1*m.pi/180
    lng1_rad = lng1*m.pi/180
    lat2_rad = lat2*m.pi/180
    lng2_rad = lng2*m.pi/180
    
    dist = 2*3959*m.asin((((m.sin((lat2_rad - lat1_rad)/2))**2) + m.cos(lat1_rad)*m.cos(lat2_rad)*((m.sin((lng2_rad - lng1_rad)/2))**2))**0.5)
    
    
    return dist

geo_df['distance_est'] = geo_df.apply(lambda x: haversine_dist(x['c_lat'],x['c_lng'],x['s_lat'],x['s_lng']), axis=1)

#Will only work w/ non-missing values for distance and geo-related analysis (441 postal codes fail to be converted to latitude/longitude by USZipCode database)
print(len(geo_df[geo_df['distance_est'].isnull()]), ' missing values.')
geo_df_nn = geo_df[geo_df['distance_est'].notna()]


fig = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(111)

#Histogram of haversine distance values
ax1.hist(geo_df_nn['distance_est'], range=(0,2000), bins=10, edgecolor='black', linewidth=1.2)

mean_dist = geo_df_nn['distance_est'].mean()
median_dist = geo_df_nn['distance_est'].median()
ax1.axvline(x=mean_dist,c='red',label='Mean = ' + str(int(round(mean_dist,1))) + ' miles',)
ax1.axvline(x=median_dist,c='green',label='Median = ' + str(int(round(median_dist,1))) + ' miles')
ax1.set_ylabel('# of \n shipments',rotation=0, labelpad=50)
ax1.set_xlabel('Estimated Distance Traveled (as crow flies, miles)')
ax1.set_title('Shipment Estimated Travel Distance Distribution, zoomed in')
ax1.set_xticks([0,200,400,600,800,1000,1200,1400,1600,1800,2000])
ax1.set_xlim(0,2000)
ax1.legend()
print('Mean distance traveled (as crow flies, miles): ', mean_dist)
print('Median distance traveled (as crow flies, miles): ', median_dist)
print("Minimum distance traveled (as crows flies, miles): ", geo_df['distance_est'].min())
print("Maximum distance traveled (as crows flies, miles): ", geo_df['distance_est'].max())
print('Number of shipments greater than 2000 miles (as the crow flies, miles): ', len(geo_df[geo_df['distance_est'] > 2000]))

plt.savefig('images/Shipment_distance_histogram.JPG',bbox_inches='tight')
plt.show()

In [ ]:
# Who are the customers placing the most orders?

q3 = '''
SELECT
    c.customer_unique_id,
    SUM(oim.price)+SUM(oim.freight_value) total_spent,
    COUNT(om.order_id) num_items,
    COUNT(DISTINCT(om.order_id)) num_orders
FROM orders_modified om
INNER JOIN order_items_modified oim ON oim.order_id = om.order_id
LEFT JOIN customers c ON c.customer_id = om.customer_id
LEFT JOIN geolocation g ON g.geolocation_zip_code = c.customer_zip_code_prefix
LEFT JOIN sellers s ON s.seller_id = oim.seller_id
LEFT JOIN geolocation g2 ON g2.geolocation_zip_code = s.seller_zip_code_prefix
GROUP BY 1
ORDER BY 4 DESC
LIMIT 10
'''

run_query(q3)

In [ ]:
# Lets look at the shipment paths for the customer who placed the most orders (16)

q4 = '''
SELECT
    c.customer_unique_id,
    g.geolocation_lat c_lat,
    g.geolocation_lng c_lng,
    c.customer_city,
    om.order_id,
    oim.seller_id,
    g2.geolocation_lat s_lat,
    g2.geolocation_lng s_lng,
    s.seller_city
FROM orders_modified om
INNER JOIN order_items_modified oim ON oim.order_id = om.order_id
LEFT JOIN customers c ON c.customer_id = om.customer_id
LEFT JOIN geolocation g ON g.geolocation_zip_code = c.customer_zip_code_prefix
LEFT JOIN sellers s ON s.seller_id = oim.seller_id
LEFT JOIN geolocation g2 ON g2.geolocation_zip_code = s.seller_zip_code_prefix
WHERE c.customer_unique_id IN ("8d50f5eadf50201ccdcedfb9e2ac8455")
'''

run_query(q4)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
map_customer1 = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)

map_customer1.drawstates(linewidth=0.5)
map_customer1.drawcoastlines(linewidth=0.5)
map_customer1.drawcountries(linewidth=1)


df_cust1 = run_query(q4)

origin_x, origin_y = map_customer1(list(df_cust1['s_lng']), list(df_cust1['s_lat']))
destination_x, destination_y = map_customer1(list(df_cust1['c_lng']),list(df_cust1['c_lat']))


# Use numpy.c_ to group arrays into two sets of coordinates: origin coordinates and destination coordinates
origin_destination_pairs = np.c_[origin_x, origin_y, destination_x, destination_y].reshape(len(origin_x), 2, 2)

# Use LineCollection to plot multiple lines on each graph.
# Use add_collection() to add line collection
# gca() method uses current axes  (ax1 in this case)
plt.gca().add_collection(LineCollection(origin_destination_pairs, color="crimson"))

map_customer1.plot(origin_x, origin_y, marker="o", color='orange', linestyle ='', label="Seller")
map_customer1.plot(destination_x, destination_y, marker="o", color='blue', linestyle ='', label="Customer")

ax1.set_title('Shipping Paths for Most Loyal Customer')
ax1.legend()

plt.savefig('images/loyal_customer_map.JPG',bbox_inches='tight')
plt.show()

loyal_customer_df = geo_df[geo_df['customer_unique_id']=='8d50f5eadf50201ccdcedfb9e2ac8455']
#print(loyal_customer_df)

loyal_customer_dist_mean = loyal_customer_df['distance_est'].mean()
loyal_customer_dist_median = loyal_customer_df['distance_est'].median()
print('Mean distance (miles): ', loyal_customer_dist_mean)
print('Median distance (miles): ', loyal_customer_dist_median)

The most loyal customer (highest # of orders of 16) is located in Sao Paulo, Sao Paulo. Their shipments came from sellers in mostly in Sao Paulo, but also Parana and Santa Catarina. The estimated shipment travel distance varied from ~3 miles to ~310 miles. The mean and median estimated shipment travel distance was ~105 miles and ~4 miles, respectively. 

In [ ]:
# Lets look at the shipment path for the shipment that traveled the furthest

fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
map_long_ship = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)

map_long_ship.drawstates(linewidth=0.5)
map_long_ship.drawcoastlines(linewidth=0.5)
map_long_ship.drawcountries(linewidth=1)

long_ship_df = geo_df[geo_df['distance_est']>2111]

origin_x, origin_y = map_long_ship(list(long_ship_df['s_lng']), list(long_ship_df['s_lat']))
destination_x, destination_y = map_long_ship(list(long_ship_df['c_lng']),list(long_ship_df['c_lat']))


# Use numpy.c_ to group arrays into two sets of coordinates: origin coordinates and destination coordinates
origin_destination_pairs = np.c_[origin_x, origin_y, destination_x, destination_y].reshape(len(origin_x), 2, 2)

# Use LineCollection to plot multiple lines on each graph.
# Use add_collection() to add line collection
# gca() method uses current axes  (ax1 in this case)
plt.gca().add_collection(LineCollection(origin_destination_pairs, color="crimson"))



map_long_ship.plot(origin_x, origin_y, marker="o", color='orange', linestyle ='', label="Seller")
map_long_ship.plot(destination_x, destination_y, marker="o", color='blue', linestyle ='', label="Customer")

ax1.set_title('Shipping Paths for Longest Shipments')
ax1.legend()

plt.savefig('images/long_ship_map.JPG',bbox_inches='tight')
plt.show()

In [ ]:
long_ship_df

The longest shipment was 2112 miles from Fazenda Rio Grande, PR to Boa Vista, RR

### Urban vs Rural Classification

Using 'BRAZIL_CITIES.csv' dataset to classify each customer and seller's location as urban or rural: The 'Brazil_CITIES.csv' dataset contains information on the 5,573 municipalities in Brazil, including their longitude & latitude coordinates. The dataset can be found here: https://www.kaggle.com/crisparada/brazilian-cities. The dataset was compiled from a variety of sources but mainly the IBGE (Brazilian Institute of Geography and Statistics). There is a column in this dataset that marks each municipality as urban or not according to the IBGE. Filtered dataset down to the 1,456 municipalities that are classified as urban by the IBGE. 

Built function (urban_feature_function) that will match each row in the geolocation table (each row corresponds to a unique zip code) to the nearest urban municipality (calling these cities). The function also calculates the distance to this city, and if it is close enough to this city (within 20 miles), whether this zip code is in an urban area or not. The function works by using my distance function that I created earlier in this section. For each zip code in the geolocation table, the distance to each of the 1,456 cities is calculated. The closest urban city is found by finding the city with the minimum distance. This process is repeated for all 19,000+ unique zip codes in the geolocation table. 

These new features are then added to the geolocation dataframe. This data is exported as a csv file and then inserted into a new table, called geolocation_updated.

Now for each customer and seller, the nearest urban city, the distance to that city and whether their location is urban or not is now known. This information can be used for each order as well: Whether the shipment was from one urban location to another urban location, whether each shipment was from a rural location to a rural location etc. Also whether the shipment was within the same urban location. 

<b> Problem: " The criteria used by the IBGE (Brazilian Institute of Geography and Statistics)[2] in determining whether households are urban or rural, however, are based on political divisions, not on the built environment." https://en.wikipedia.org/wiki/List_of_largest_cities_in_Brazil#cite_note-WorldBank-1 

In [ ]:
cities_df = pd.read_csv('BRAZIL_CITIES.csv',delimiter=';')
cities_df = cities_df.sort_values(by='IBGE_RES_POP', ascending=False)


In [ ]:
urban_df = cities_df[cities_df['RURAL_URBAN']=='Urbano']
urban_df.reset_index(drop=True, inplace=True)
print('Number of municipalties in this dataset: ', len(cities_df))
print('Number of municipalities classified as urban in this dataset: ', len(urban_df))
urban_df.head(5)

In [ ]:
#Updated map

from mpl_toolkits.basemap import Basemap
from matplotlib.collections import LineCollection

fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(121)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
#map_customers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=80, llcrnrlon=-130, urcrnrlon=30)
map_urban = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)

map_customers.drawstates(linewidth=0.5)
map_customers.drawcoastlines(linewidth=0.5)
map_customers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
urban_x, urban_y = map_customers(list(urban_df['LONG']), list(urban_df['LAT']))

#3. Plot cartesian coordinates
ax1.scatter(urban_x, urban_y,s=3)
ax1.set_title('Urban Locations')

plt.show()

#Note that some of the locations are still off: the location that looks to be in Argentina is actually Santa Rosa which is firmly in Brazil. Another case of wrong lat/lng values.

In [ ]:
print(len(urban_df[urban_df['LAT'].isnull()]))
print(len(urban_df[urban_df['LONG'].isnull()]))

In [ ]:
q5 = '''
SELECT *
FROM geolocation
'''

geolocation_table = run_query(q5)

In [ ]:
### Function that creates feature "urban":

# Apply function to each row in the dataframe. For each row:
# Loop through each row in urban_df. Calculate Haversine Distance. If Haversine Distance < 25 miles, set urban_df value to 1 and end loop
# Else, keep going through each row in urban_df. If the customer/seller isn't within 25 miles of any of the 1,500 urban areas, it will have a value of 0 for rural.


def urban_feature_function(df_row):

    lat = df_row['geolocation_lat']
    lng = df_row['geolocation_lng']
    distance_series = urban_df.apply(lambda x: haversine_dist(x['LAT'],x['LONG'], lat, lng), axis=1)
    closest_index = distance_series.idxmin()
    closest_distance = distance_series[closest_index]
    closest_city = urban_df.loc[closest_index,'CITY']
    urban=0
    if closest_distance < 20:
        urban =1 

    return (closest_city, closest_distance, urban)

Note that I have commented out the cell below b/c it takes a long time to run (~940 seconds the last time it was ran) and doesn't need to be run multiple times. The result was inserted into the file 'geolocation_updated.csv'. And then inserted into a table.

In [ ]:
#import time
#start = time.time()

#urban_series = geolocation_table.apply(lambda x: urban_feature_function(x), axis=1)

#urban_df = pd.DataFrame(urban_series.tolist(), index=geolocation_table.index).rename(columns={0:'Closest_Urban_City', 1:'Urban_City_Distance', 2:'Urban'}) 
#geolocation_table_updated = pd.concat([geolocation_table, urban_df], axis=1)

#geolocation_table_updated.to_csv('data/brazilian-ecommerce/geolocation_updated.csv', index=False)

#end = time.time()
#time_amount = end - start
#print(time_amount, ' seconds.')

In [ ]:
#geolocation_table_updated.head(10)

In [ ]:
geolocation_table_updated = pd.read_csv('data/brazilian-ecommerce/geolocation_updated.csv')

geolocation_table_updated.info()

<b> Create new, updated geolocation table w/ the additional fields Closest_Urban_City, Urban_City_Distance and Urban</b>:<br>
Create geolocation_updated table: 
![title](images/geolocation_updated.JPG)

Insert data into geolocation_updated table:
![title](images/insert_geolocation_updated.JPG)


<b>Added new foreign key to sellers table </b>: <br>

<b>Special note(s)</b>: 
1. SQLite lacks the functionality to alter existing table and add foreign key. Have to add foreign key to table the long way (see below).
2. Ran commands through run_command function rather than through DB Browser b/c the run_command function does a much better job of managing the connection to the database (there is only a connection to the database when the run_command function is running, similar to the run_query function). Using the DB Browser for these tasks resulted in the program either crashing or returning "data base disk image is malformed".
![title](images/run_command_function.JPG)
3. Note that creating a new foreign key for the customers and sellers tables to the new geolocation_updated table is not an absolute necessity: can join non-foreign key to non-primary key fields but w/o these constraints, not guaranteed to maintain data integrity so this would require one to be very careful.


drop table sellers:
![title](images/drop_table_sellers_final.JPG)
create table sellers w/ new foreign key:
![title](images/create_table_sellers_new_fk_final.JPG) 
insert sellers data from csv:
![title](images/insert_data_sellers.JPG) 
drop header row from sellers table:
![title](images/sellers_drop_header_final.JPG) 


<b>Added new foreign key to customers table</b>: <br>

drop table customers:
![title](images/drop_table_customers.JPG) 
create table customers w/ new foreign key:
![title](images/create_table_customers_new_fk.JPG) 
insert customers data from csv:
![title](images/insert_data_customers.JPG) 
drop header row from customers table:
![title](images/delete_header_customers.JPG)
 

In [ ]:
#Create manager to handle queries that don't return tables (ie commands, not queries) like for creating views.
# Manages connection to database such that the database is only connected when this function is running (same as run_query function)
def run_command(c):
    with sqlite3.connect('data/ecommerce.db') as conn:
        conn.isolation_level=None
        conn.execute(c)

In [ ]:
# Commented out all commands b/c only want to run commands once ever

#q_command_1 = '''
#DROP TABLE IF EXISTS sellers
#'''

#run_command(q_command_1)

In [ ]:
#q_tables = '''
#SELECT name, sql FROM sqlite_master
#WHERE type='table'
#ORDER BY name;
#'''

#run_query(q_tables)

In [ ]:
#q_command_2 = '''
#CREATE TABLE 'sellers' (
#    'seller_id' [TEXT] PRIMARY KEY,
#    'seller_zip_code_prefix' [INTEGER],
#    'seller_city' [TEXT],
#    'seller_state' [TEXT],
#    FOREIGN KEY ('seller_zip_code_prefix')
#        REFERENCES geolocation ('geolocation_zip_code'),
#    FOREIGN KEY ('seller_zip_code_prefix')
#        REFERENCES geolocation_updated ('geolocation_zip_code')
#    );
#'''

#run_command(q_command_2)

In [ ]:
# Drop header row

#q_command_3 = '''
#DELETE FROM sellers
#WHERE seller_id='seller_id';
#'''

#run_command(q_command_3)

In [ ]:
#q_command_4 = '''
#DROP TABLE IF EXISTS customers
#'''

#run_command(q_command_4)

In [ ]:
# Create customers table w/ new foreign key

#q_command_5 = '''
#CREATE TABLE 'customers' (
#    'customer_id' [TEXT] PRIMARY KEY,
#    'customer_unique_id' [TEXT],
#    'customer_zip_code_prefix' [INTEGER],
#    'customer_city' [TEXT],
#    'customer_state' [TEXT],
#    FOREIGN KEY ('customer_zip_code_prefix')
#        REFERENCES geolocation ('geolocation_zip_code'),
#    FOREIGN KEY ('customer_zip_code_prefix')
#        REFERENCES geolocation_updated ('geolocation_zip_code')
#    );
#'''

#run_command(q_command_5)

In [ ]:
# Drop header row

#q_command_6 = '''
#DELETE FROM customers
#WHERE customer_id='customer_id';
#'''

#run_command(q_command_6)

In [ ]:
# New geo_df w/ the three new urban features: 

q5 = '''
SELECT 
    c.*,
    g.geolocation_lat c_lat,
    g.geolocation_lng c_lng,
    g.Closest_Urban_City Customer_Urban_City,
    g.Urban_City_Distance Customer_Urban_Distance,
    g.Urban Customer_Urban,
    merged.*,
    s.*,
    g2.geolocation_lat s_lat,
    g2.geolocation_lng s_lng,
    g2.Closest_Urban_City Seller_Urban_City,
    g2.Urban_City_Distance Seller_Urban_Distance,
    g2.Urban Seller_Urban
FROM (
    SELECT *
    FROM orders_modified om
    LEFT JOIN order_items_modified oim ON oim.order_id=om.order_id
    GROUP BY om.order_id
    ) as merged
LEFT JOIN customers c ON c.customer_id = merged.customer_id
LEFT JOIN geolocation_updated g ON g.geolocation_zip_code = c.customer_zip_code_prefix
LEFT JOIN sellers s ON s.seller_id = merged.seller_id
LEFT JOIN geolocation_updated g2 ON g2.geolocation_zip_code = s.seller_zip_code_prefix
WHERE merged.order_status='delivered' AND length(c_lat)>0 AND length(s_lat)>0
'''

geo_df_updated = run_query(q5)
geo_df_updated

In [ ]:
print('Number of rows in geo_df: ', len(geo_df))
print('Number of rows in geo_df_updated: ', len(geo_df_updated))

print('Results of table_audit_function for geo_df:')
table_audit_function(geo_df)
print('Results of table_audit_function for geo_df_updated:')
table_audit_function(geo_df_updated)

print('\nConclusion: These two dataframes are identifical except geo_df_updated has six new urban features.')

In [ ]:
# Urban/rural location composition for customers:

q6 = '''
SELECT 
    COUNT(DISTINCT(c.customer_unique_id)) num_customers,
    gu.Urban
FROM customers c
LEFT JOIN geolocation_updated gu ON gu.geolocation_zip_code=c.customer_zip_code_prefix
GROUP BY 2
'''

run_query(q6)

# Make urban / rural / unknown pie chart

In [ ]:
customer_urban_comp = run_query(q6)
customer_urban_comp = customer_urban_comp.loc[1:]
fig = plt.figure(figsize=(6,6))
ax1 = fig.add_subplot(111)

label = ['Rural', 'Urban']

ax1.pie(x=customer_urban_comp['num_customers'], labels=label, autopct='%1.1f%%', explode=(0,0.1))
ax1.set_title("Customer Urban-Rural Composition")
plt.savefig('images/customer_urban_rural.JPG',bbox_inches='tight')
plt.show()

96.2% of the Olist customers are located in urban areas. This matches pretty closely with the overall Brazilian urban composition (by population) as of 2018 of 86.6% (Source: https://data.worldbank.org/indicator/SP.URB.TOTL.IN.ZS?locations=BR). It makes intuitive sense that the urban-rural composition would be higher than that considering that individuals in rural areas are probably less likely to use ecommerce.

In [ ]:
# Urban/rural location composition for sellers:

q7 = '''
SELECT 
    COUNT(DISTINCT(s.seller_id)) num_sellers,
    gu.Urban
FROM sellers s
LEFT JOIN geolocation_updated gu ON gu.geolocation_zip_code=s.seller_zip_code_prefix
GROUP BY 2
'''

run_query(q7)

# Make urban / rural / unknown pie chart

In [ ]:
seller_urban_comp = run_query(q7)
seller_urban_comp = seller_urban_comp.loc[1:]
fig = plt.figure(figsize=(6,6))
ax1 = fig.add_subplot(111)

label = ['Rural', 'Urban']

ax1.pie(x=seller_urban_comp['num_sellers'], labels=label, autopct='%1.1f%%', explode=(0,0.1))
ax1.set_title("Seller Urban-Rural Composition")
plt.savefig('images/seller_urban_rural.JPG',bbox_inches='tight')
plt.show()

99.2% of the Olist sellers are located in urban areas. This makes intuitive sense considering the overall Brazilian urban-rural composition (by population) as of 2018 is 86.7% and the fact that businesses are probably more likely to be in urban areas.

In [ ]:
fig = plt.figure(figsize=(13,6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

# Customers
customer_urban_comp = run_query(q6)
customer_urban_comp = customer_urban_comp.loc[1:]
label = ['Rural', 'Urban']
ax1.pie(x=customer_urban_comp['num_customers'], autopct='%1.1f%%', explode=(0,0.1))
ax1.set_title("Customers", size=20)

#Sellers
seller_urban_comp = run_query(q7)
seller_urban_comp = seller_urban_comp.loc[1:]


ax2.pie(x=seller_urban_comp['num_sellers'], autopct='%1.1f%%', explode=(0,0.1))
ax2.set_title("Sellers", size=20)
plt.savefig('images/customer_seller_urban_rural.JPG',bbox_inches='tight')
plt.legend(label)
plt.show()

In [ ]:
num_urban_urban = len(geo_df_updated[(geo_df_updated['Seller_Urban']==1) & (geo_df_updated['Customer_Urban']==1)])
print(num_urban_urban)
num_urban_rural = len(geo_df_updated[(geo_df_updated['Seller_Urban']==1) & (geo_df_updated['Customer_Urban']==0)])
print(num_urban_rural)
num_rural_urban = len(geo_df_updated[(geo_df_updated['Seller_Urban']==0) & (geo_df_updated['Customer_Urban']==1)])
print(num_rural_urban)
num_rural_rural = len(geo_df_updated[(geo_df_updated['Seller_Urban']==0) & (geo_df_updated['Customer_Urban']==0)])
print(num_rural_rural)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax1 = fig.add_subplot(111)

urban_comp_shipments = [num_urban_urban, num_urban_rural, num_rural_urban, num_rural_rural]
label = ['Urban Seller to Urban Customer', 'Urban Seller to Rural Customer', 'Rural Seller to Urban Customer', 'Rural Seller to Rural Customer']

ax1.pie(x=urban_comp_shipments , labels=label, autopct='%1.1f%%', explode=(0,0,0,2))
ax1.set_title("Urban-Rural Shipment Composition")
plt.savefig('images/urban_shipments.JPG',bbox_inches='tight')
plt.show()

The vast majority (96.0%) of the delivered shipments are from an urban seller to an urban customer. <br>
3.8% of the delivered shipments are from an urban seller to a rural customer. <br>
Rural to urban and rural to rural delivered shipments are virtually non-existent (0.2% and 0.008%, respectively).

In [ ]:
#Updated map for urban vs rural classification for customers and sellers.

from mpl_toolkits.basemap import Basemap
from matplotlib.collections import LineCollection

fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(121)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor

map_customers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)

map_customers.drawstates(linewidth=0.5)
map_customers.drawcoastlines(linewidth=0.5)
map_customers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
urban_customers_x, urban_customers_y = map_customers(list(geo_df_updated.loc[geo_df_updated['Customer_Urban']==1,'c_lng']), list(geo_df_updated.loc[geo_df_updated['Customer_Urban']==1,'c_lat']))
rural_customers_x, rural_customers_y = map_customers(list(geo_df_updated.loc[geo_df_updated['Customer_Urban']==0,'c_lng']), list(geo_df_updated.loc[geo_df_updated['Customer_Urban']==0,'c_lat']))


#3. Plot cartesian coordinates
ax1.scatter(urban_customers_x, urban_customers_y, s=2,c='tab:purple')
ax1.scatter(rural_customers_x, rural_customers_y,s=2, c='tab:green')

ax1.set_title('Customer Locations')
ax2 = fig.add_subplot(122)

#1. create a new basemap instance with the specific map projection we want and how much of the map we want included:
#Using the Basemap() constructor
map_sellers = Basemap(projection='merc', llcrnrlat=-60, urcrnrlat=20, llcrnrlon=-85, urcrnrlon=-30)
map_sellers.drawstates(linewidth=0.5)
map_sellers.drawcoastlines(linewidth=0.5)
map_sellers.drawcountries(linewidth=1)

#2. convert spherical coordinates to cartesian coordinates using the basemap instance
urban_sellers_x, urban_sellers_y = map_customers(list(geo_df_updated.loc[geo_df_updated['Seller_Urban']==1,'s_lng']), list(geo_df_updated.loc[geo_df_updated['Seller_Urban']==1,'s_lat']))
rural_sellers_x, rural_sellers_y = map_customers(list(geo_df_updated.loc[geo_df_updated['Seller_Urban']==0,'s_lng']), list(geo_df_updated.loc[geo_df_updated['Seller_Urban']==0,'s_lat']))

#3. Plot cartesian coordinates
ax2.scatter(urban_sellers_x, urban_sellers_y, s=2,c='tab:purple',label='Urban')
ax2.scatter(rural_sellers_x, rural_sellers_y,s=2, c='tab:green',label='Rural')

ax2.set_title('Seller Locations')
ax2.legend(markerscale=8)
plt.savefig('images/customer_seller_locations_urban.JPG',bbox_inches='tight')
plt.show()

In [ ]:
num_same = len(geo_df_updated[(geo_df_updated['Seller_Urban_City']==geo_df_updated['Customer_Urban_City'])&(geo_df_updated['Customer_Urban']==1)&(geo_df_updated['Seller_Urban']==1)])
print('Number of shipments that stay within the same metropolitan area:',num_same)
# 1,529 of the shipments occur within the same urban city.

#Do same analysis for the 28 metropolitan areas??

metropolitan_areas_list = [São Paulo, Rio de Janeiro, Belo Horizonte, Brasilia, Porto Alegre, Fortaleza	 Ceará	4,051,744	4,019,213	+0.8%
7	Salvador	 Bahia	4,015,205	3,984,583	+0.8%
8	Recife	 Pernambuco	3,965,699	3,940,456	+0.6%
9	Curitiba	 Paraná	3,572,326	3,537,894	+1.0%
10	Campinas	 São Paulo	3,168,019	3,131,528	+1.2%
11	Manaus	 Amazonas	2,612,747	2,568,817	+1.7%
12	Vale do Paraíba e Litoral Norte (São José dos Campos)	 São Paulo	2,497,857	2,475,879	+0.9%
13	Goiânia	 Goiás	2,493,792	2,458,504	+1.4%
14	Belém	 Pará	2,441,761	2,422,481	+0.8%
15	Sorocaba	 São Paulo	2,088,381	1,908,425	+9.4%
16	Vitória	 Espírito Santo	1,960,213	1,935,483	+1.3%
17	Baixada Santista (Santos)	 São Paulo	1,828,212	1,813,033	+0.8%
18	Ribeirão Preto	 São Paulo	1,678,910	1,662,645	+1.0%
19	São Luís	 Maranhão	1,619,377	1,605,305	+0.9%
20	Natal	 Rio Grande do Norte	1,596,104	1,537,211	+3.8%
21	Piracicaba	 São Paulo	1,464,993	1,452,691	+0.8%
22	Norte/Nordeste Catarinense (Joinville)	 Santa Catarina	1,383,456	1,363,854	+1.4%
23	Maceió	 Alagoas	1,352,241	1,314,254	+2.9%
24	João Pessoa	 Paraíba	1,282,227	1,268,360	+1.1%
25	Teresina	 Piauí / Maranhão	1,204,397	1,199,941	+0.4%
26	Florianópolis	 Santa Catarina	1,172,076	1,152,115	+1.7%
27	Londrina	 Paraná	1,094,347	1,085,479	+0.8%
28	Vale do Rio Cuiabá (Cuiabá)

In [ ]:
print(len(urban_df['REGIAO_TUR'].unique()))

In [ ]:
# Now do for ecommerce.db (save ecommerce.db copy first in separate folder)
# Do in new file
# Document every single step along the way w/ snippets. Paste all snippets into markdown above.
# End result is now sellers and customers can join geolocation_updated while preserving data integrity.

# Will have to do same procedure when do new features like google maps API distance.